## **UAOPiloto**
Detección de estudiantes que van a desertar en 2 períodos.

### **1.  Conexión a la base de datos**
Nos conectamos a la base de datos MYSQL

In [2]:
import pandas as pd
from sqlalchemy import create_engine

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)

### **2.  Detección estudiantes que han desertado**
Se detecta qué estudiante ha dejado de matricular 2 periodos seguidos sin ser que se ha graduado

2.1 DATOS:  la tabla base de generación de predicciones.   DATOS(id_estudiante, periodo)
2.2 PERIODOS:  son los períodos regulares.  PERIODOS(periodo, orden).  Orden nos señala en qué orden se presentan los períodos.

A Datos le adicionamos el primer periodo que hizo matrícula cada estudiante
PRIMER_PERIODO (id_estudiante, periodo), donde periodo es el primer periodo. 

In [25]:
query = """
    SELECT distinct id_estudiante, periodo
    FROM estudianteAsignatura
    ORDER BY id_estudiante, periodo
"""
DATOS = pd.read_sql(query, con=motor)



# NOTA:  falta manejar el caso de cursos de verano...  cuándo marcar al estudainte como desertor?  ERIK ya tiene la pregunta 20231009

In [35]:
# adicionamos el primer periodo en que ese estudiante hizo una matricula
query = """
    SELECT  id_estudiante, min(periodo) periodo_inicial
    FROM EstudiantesMatriculas
    WHERE periodo like '%1' or periodo like '%3'
    GROUP BY id_estudiante
"""
temp = pd.read_sql(query, con=motor)
# hacemos join entre DATOS y PRIMER_PERIODO para adicionar el periodo_inicial
DATOS2 = DATOS.merge(temp, on=['id_estudiante'], how='left')

In [34]:
# ahora adicionamos el periodo de grado Y TODAS LAS COLUMNAS REQUERIDAS DESDE ESTUDIANTES_LIMPIO pues ya hice todo el trabajo
# de limpieza.

query = """
    SELECT codigo id_estudiante, fecha_grado  <INCLUIR TODAS LAS COLUMNAS NECESARIAS>
    FROM estudiantes_limpio
"""
temp = pd.read_sql(query, con=motor)

# hacemos join con DATOS para adicionar la fecha de grado
DATOS3 = DATOS2.merge(temp, on=['id_estudiante'], how='left')





In [33]:
DATOS3

,id_estudiante,periodo,fecha_grado_x,fecha_grado_y
0,2136689,201803,NaT,NaT
1,2136689,201803,NaT,NaT
2,2136689,201803,NaT,NaT
3,2136689,201803,NaT,NaT
4,2136689,201803,NaT,NaT
...,...,...,...,...
61842,2236079,202303,NaT,NaT
61843,2236229,202303,NaT,NaT
61844,2236496,202303,NaT,NaT
61845,2236497,202303,NaT,NaT
